In [1]:
using Apophis, BenchmarkTools
using Symbolics: build_function, jacobian, derivative, Num, @variables

In [2]:
# P₀ = only(@variables P₀)
# T = only(@variables T)
# Y = collect(only(@variables Y[1:10]));
# T₀ = only(@variables T₀)
# Y₀ = collect(only(@variables Y₀[1:10]));

In [3]:
init(:H2, 1000.0, 1.59e6, H2 = 0.29, N2 = 0.56, O2 = 0.15)
#init(:GRI3, 1000.0, CH4 = 0.5, N2 = 0.75, O2 = 0.20)
#init(:H2, T₀, P₀, Y₀);

In [4]:
## Symbolically
#step!(gas, Y₀, T₀)
#step!(gas, Y, T)

## Numerically
step!(gas, gas.initial.mass_fractions, gas.initial.temperature)
#step!(gas, gas.current.mass_fractions, gas.current.temperature)

### $$ A = \begin{bmatrix} \bm{a} & \bm{b}\\ \bm{c} &d \end{bmatrix} $$ 

$$ \bm{a} = \frac{∂\bm{Ẏ}}{∂\bm{Y}}\ \ [ns\times ns], \ \ \bm{b} = \frac{d\bm{Ẏ}}{dT}\ \ [ns\times 1]$$
$$ \bm{c} = \frac{d\bm{Ẏ}}{dT} \ \ [1\times ns], \ \ d =  \frac{dṪ}{dT}\ \ [1\times 1]$$

In [7]:
dṪ = only(gas.intermediate.temperature_change_rate)
dẎ = gas.intermediate.mass_change_rate
q = vcat(dẎ, dṪ)
u = vcat(Y, T)

A = jacobian(q, u);
_, A_expr = build_function(A, Y, T, Y₀, T₀, P₀)
A_function = eval(A_expr);

In [18]:
#A = zeros(11, 11)
#A_function(A, ones(10), 1200.0, ones(10), 1300.0, 1000)

  0.001944 seconds (2 allocations: 288 bytes)


In [8]:
open("./jac/H2.jl", "a") do io
   println(io, A_expr)
   #replace(io, "(ˍ₋out, ˍ₋arg1, T, ˍ₋arg3, T₀, P₀)" => "jac(ˍ₋out, ˍ₋arg1, T, ˍ₋arg3, T₀, P₀)")
end

### <center>Jacobian Trials</center>

In [5]:
A = zeros(11, 11)

11×11 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [7]:
include("/Users/sabry/Developer/Apophis/jac/H2.jl")

jac (generic function with 1 method)

In [8]:
jac(A, gas.initial.mass_fractions, gas.initial.temperature, gas.initial.mass_fractions, gas.initial.temperature, gas.initial.pressure)